to store secrets locally because I don't know how else

In [1]:
%load_ext dotenv
%dotenv 

In [2]:
import os

In [3]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("./dataset/raw_html")
documents = loader.load()

In [6]:
for doc in documents:
    doc.metadata["filename"] = doc.metadata["source"]

In [11]:
# get first 10 for testing i guess
ndocuments = documents[:10]
ndocuments

[Document(metadata={'source': 'dataset\\raw_html\\(N_A)Network_VPN_OSX.html', 'filename': 'dataset\\raw_html\\(N_A)Network_VPN_OSX.html'}, page_content='Network/VPN/macOS\n\nFrom UMIACS\n\n(Redirected from (N/A)Network/VPN/OSX )\n\nJump to navigation Jump to search\n\nThe UMIACS VPN is accessible through the Ivanti Connect Secure Client. Alternatively, you can establish a connection through a web browser , but this may fail if you have an outdated version of Ivanti Connect Secure already installed.\n\nContents\n\n1 Prerequisites\n\n2 Connecting through the Ivanti Connect Secure client\n\n3 Checking VPN Status\n\n4 Setup Security Requirement (Follow these steps if you encounter an error while connecting through the Ivanti Secure client)\n\nPrerequisites\n\nmacOS 13.x or higher as of the current version of the Ivanti Connect Secure Client\n\nA full UMIACS account\n\nDevice enrolled with UMIACS Duo for multi-factor authentication\n\nAn established network connection (wired or wireless)\n\

In [12]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-4o-mini")
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings 
)

In [13]:
testset = generator.generate_with_langchain_docs(ndocuments, test_size=10, distributions={simple: 0.5, reasoning: 0.5, multi_context: 0})

Generating: 100%|██████████| 10/10 [02:11<00:00, 13.10s/it]     


In [16]:
df = testset.to_pandas()
df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,"What types of images can Apptainer run, and ho...",[Apptainer\n\nFrom UMIACS\n\nJump to navigatio...,Apptainer can run a variety of images includin...,simple,"[{'source': 'dataset\raw_html\Apptainer.html',...",True
1,What are the guidelines for installing and lic...,[Adobe\n\nFrom UMIACS\n\nJump to navigation Ju...,The guidelines for installing and licensing Ad...,simple,"[{'source': 'dataset\raw_html\Adobe.html', 'fi...",True
2,What steps should you follow to change your UM...,[Accounts\n\nFrom UMIACS\n\nJump to navigation...,To change your UMIACS account password if you ...,simple,"[{'source': 'dataset\raw_html\Accounts.html', ...",True
3,What is the minimum macOS version required to ...,[Network/VPN/macOS\n\nFrom UMIACS\n\n(Redirect...,The minimum macOS version required to use the ...,simple,[{'source': 'dataset\raw_html\(N_A)Network_VPN...,True
4,What is the purpose of binding the Nexus scrat...,[^3\r\n\nBind Mounts\n\nTo get data into the c...,The purpose of binding the Nexus scratch direc...,simple,"[{'source': 'dataset\raw_html\Apptainer.html',...",True
5,Which command allows GPU access for CUDA in co...,[\n2022/10/14 10:31:20 info unpack layer: sha...,The command that allows GPU access for CUDA in...,reasoning,"[{'source': 'dataset\raw_html\Apptainer.html',...",True
6,How does the GTX 1080 Ti assist with CUDA in c...,[\n2022/10/14 10:31:20 info unpack layer: sha...,The answer to given question is not present in...,reasoning,"[{'source': 'dataset\raw_html\Apptainer.html',...",True
7,What should a UMIACS password avoid to not mat...,[Accounts\n\nFrom UMIACS\n\nJump to navigation...,A UMIACS password should not contain (or match...,reasoning,"[{'source': 'dataset\raw_html\Accounts.html', ...",True
8,Which flag is key for GPU access in CUDA conta...,[Apptainer\n\nFrom UMIACS\n\nJump to navigatio...,The answer to given question is not present in...,reasoning,"[{'source': 'dataset\raw_html\Apptainer.html',...",True
9,How to install a UMIACS cert in Windows browsers?,[AddingUMIACSCertificateAuthority\n\nFrom UMIA...,To install a UMIACS certificate in Windows bro...,reasoning,[{'source': 'dataset\raw_html\AddingUMIACSCert...,True


In [36]:
print(df["ground_truth"][7], "\n")

A UMIACS password should not contain (or match a substring of 3+ sequential characters) of your username or any part of your first or last name. 



this is terrible the original context is this: 

<h2><span class="mw-headline" id="Password_Requirements">Password Requirements</span></h2>
<p>We have the following policies in regards to our <a href="/umiacs/index.php/ActiveDirectory" title="ActiveDirectory">ActiveDirectory</a> passwords:
</p>
<ul><li>Minimum Password Length&nbsp;: 8 Characters (more is allowed)</li>
<li>Minimum Character Classes&nbsp;: 3 Character Classes
<ul><li>English uppercase characters (A - Z)</li>
<li>English lowercase characters (a - z)</li>
<li>Base 10 digits (0 - 9)</li>
<li>Non-alphanumeric (For example:&nbsp;!, $, #, or&nbsp;%)</li>
<li>Unicode characters</li></ul></li>
<li>Password cannot contain (or match a substring of 3+ sequential characters) of your username or any part of your first or last name.</li>
<li>Password was not previously used in last 5 passwords.</li>
<li>Password has not been <a href="/umiacs/index.php/CompromisedPasswordFiltering" title="CompromisedPasswordFiltering"> compromised</a> in a known data breach.</li></ul>
<p>It is your responsibility to secure your password. Please choose a strong password, do not use your UMIACS password for any other service, and only connect using secure protocols.
</p><p>UMIACS staff members do not store your password in a retrievable format, so if you forget your password, you must reset it (next section).
</p>

In [52]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
models = __import__("model wrappers") # it's not letting me rename and i don't wanna rerun this notebook
RAGFlow = models.wrappers.RAGFlow

In [78]:
chat = RAGFlow("http://localhost", "ragflow-FlMGI3NDYyODExYjExZWZiZmJjMDI0Mm")

In [79]:
chat("hi")

'The answer you are looking for is not found in the knowledge base.'

In [81]:
x = df.apply(lambda x: chat(x["question"]), axis=1)

In [85]:
df["answer"] = x
df

,question,contexts,ground_truth,evolution_type,metadata,episode_done,answer
0,"What types of images can Apptainer run, and ho...",[Apptainer\n\nFrom UMIACS\n\nJump to navigatio...,Apptainer can run a variety of images includin...,simple,"[{'source': 'dataset\raw_html\Apptainer.html',...",True,Apptainer can run a variety of images includin...
1,What are the guidelines for installing and lic...,[Adobe\n\nFrom UMIACS\n\nJump to navigation Ju...,The guidelines for installing and licensing Ad...,simple,"[{'source': 'dataset\raw_html\Adobe.html', 'fi...",True,"At UMIACS, the commonly deployed Adobe product..."
2,What steps should you follow to change your UM...,[Accounts\n\nFrom UMIACS\n\nJump to navigation...,To change your UMIACS account password if you ...,simple,"[{'source': 'dataset\raw_html\Accounts.html', ...",True,"If you know your current password, you can use..."
3,What is the minimum macOS version required to ...,[Network/VPN/macOS\n\nFrom UMIACS\n\n(Redirect...,The minimum macOS version required to use the ...,simple,[{'source': 'dataset\raw_html\(N_A)Network_VPN...,True,The minimum macOS version required to use the ...
4,What is the purpose of binding the Nexus scrat...,[^3\r\n\nBind Mounts\n\nTo get data into the c...,The purpose of binding the Nexus scratch direc...,simple,"[{'source': 'dataset\raw_html\Apptainer.html',...",True,"To get data into the Apptainer container, you ..."
5,Which command allows GPU access for CUDA in co...,[\n2022/10/14 10:31:20 info unpack layer: sha...,The command that allows GPU access for CUDA in...,reasoning,"[{'source': 'dataset\raw_html\Apptainer.html',...",True,The command that allows GPU access for CUDA in...
6,How does the GTX 1080 Ti assist with CUDA in c...,[\n2022/10/14 10:31:20 info unpack layer: sha...,The answer to given question is not present in...,reasoning,"[{'source': 'dataset\raw_html\Apptainer.html',...",True,The GTX 1080 Ti assists with CUDA in container...
7,What should a UMIACS password avoid to not mat...,[Accounts\n\nFrom UMIACS\n\nJump to navigation...,A UMIACS password should not contain (or match...,reasoning,"[{'source': 'dataset\raw_html\Accounts.html', ...",True,The knowledge base does not provide specific d...
8,Which flag is key for GPU access in CUDA conta...,[Apptainer\n\nFrom UMIACS\n\nJump to navigatio...,The answer to given question is not present in...,reasoning,"[{'source': 'dataset\raw_html\Apptainer.html',...",True,The key flag for accessing GPUs in CUDA contai...
9,How to install a UMIACS cert in Windows browsers?,[AddingUMIACSCertificateAuthority\n\nFrom UMIA...,To install a UMIACS certificate in Windows bro...,reasoning,[{'source': 'dataset\raw_html\AddingUMIACSCert...,True,"For most Windows browsers (Chrome, Firefox, Ed..."


In [93]:
from ragas.metrics import (answer_relevancy, FaithulnesswithHHEM, context_recall, context_precision)
faithfulness = FaithulnesswithHHEM() # why is there a typo xd ? 
from ragas import evaluate

c:\Users\Andrew\miniforge3\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Andrew\.cache\huggingface\hub\models--vectara--hallucination_evaluation_model. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
A new version of the following files was downloaded from https://huggingface.co/vectara/hallucination_

In [95]:
from datasets import Dataset

In [96]:
dataset = Dataset.from_pandas(df)

In [97]:
result = evaluate(dataset, metrics=[context_precision, faithfulness, answer_relevancy, context_recall])
result

Evaluating: 100%|██████████| 40/40 [00:57<00:00,  1.43s/it]


{'context_precision': 0.8000, 'faithfulness_with_hhem': 0.7091, 'answer_relevancy': 0.8535, 'context_recall': 1.0000}

In [104]:
resultdf = result.to_pandas()
resultdf

,question,contexts,ground_truth,evolution_type,metadata,episode_done,answer,context_precision,faithfulness_with_hhem,answer_relevancy,context_recall
0,"What types of images can Apptainer run, and ho...",[Apptainer\n\nFrom UMIACS\n\nJump to navigatio...,Apptainer can run a variety of images includin...,simple,[{'filename': 'dataset\raw_html\Apptainer.html...,True,Apptainer can run a variety of images includin...,1.0,0.666667,0.957453,1.0
1,What are the guidelines for installing and lic...,[Adobe\n\nFrom UMIACS\n\nJump to navigation Ju...,The guidelines for installing and licensing Ad...,simple,"[{'filename': 'dataset\raw_html\Adobe.html', '...",True,"At UMIACS, the commonly deployed Adobe product...",1.0,0.875000,0.963876,1.0
2,What steps should you follow to change your UM...,[Accounts\n\nFrom UMIACS\n\nJump to navigation...,To change your UMIACS account password if you ...,simple,[{'filename': 'dataset\raw_html\Accounts.html'...,True,"If you know your current password, you can use...",1.0,1.000000,0.878369,1.0
3,What is the minimum macOS version required to ...,[Network/VPN/macOS\n\nFrom UMIACS\n\n(Redirect...,The minimum macOS version required to use the ...,simple,[{'filename': 'dataset\raw_html\(N_A)Network_V...,True,The minimum macOS version required to use the ...,1.0,1.000000,0.999992,1.0
4,What is the purpose of binding the Nexus scrat...,[^3\r\n\nBind Mounts\n\nTo get data into the c...,The purpose of binding the Nexus scratch direc...,simple,[{'filename': 'dataset\raw_html\Apptainer.html...,True,"To get data into the Apptainer container, you ...",1.0,1.000000,0.830004,1.0
5,Which command allows GPU access for CUDA in co...,[\n2022/10/14 10:31:20 info unpack layer: sha...,The command that allows GPU access for CUDA in...,reasoning,[{'filename': 'dataset\raw_html\Apptainer.html...,True,The command that allows GPU access for CUDA in...,1.0,0.500000,0.997138,1.0
6,How does the GTX 1080 Ti assist with CUDA in c...,[\n2022/10/14 10:31:20 info unpack layer: sha...,The answer to given question is not present in...,reasoning,[{'filename': 'dataset\raw_html\Apptainer.html...,True,The GTX 1080 Ti assists with CUDA in container...,0.0,0.818182,1.000000,1.0
7,What should a UMIACS password avoid to not mat...,[Accounts\n\nFrom UMIACS\n\nJump to navigation...,A UMIACS password should not contain (or match...,reasoning,[{'filename': 'dataset\raw_html\Accounts.html'...,True,The knowledge base does not provide specific d...,1.0,0.230769,0.000000,1.0
8,Which flag is key for GPU access in CUDA conta...,[Apptainer\n\nFrom UMIACS\n\nJump to navigatio...,The answer to given question is not present in...,reasoning,[{'filename': 'dataset\raw_html\Apptainer.html...,True,The key flag for accessing GPUs in CUDA contai...,0.0,0.000000,0.971190,1.0
9,How to install a UMIACS cert in Windows browsers?,[AddingUMIACSCertificateAuthority\n\nFrom UMIA...,To install a UMIACS certificate in Windows bro...,reasoning,[{'filename': 'dataset\raw_html\AddingUMIACSCe...,True,"For most Windows browsers (Chrome, Firefox, Ed...",1.0,1.000000,0.937092,1.0


In [114]:
resultdf[["question", "contexts", "ground_truth", "answer", "context_precision", "faithfulness_with_hhem"]]

,question,contexts,ground_truth,answer,context_precision,faithfulness_with_hhem
0,"What types of images can Apptainer run, and how does it relate to Docker images?","[Apptainer\n\nFrom UMIACS\n\nJump to navigation Jump to search\n\nApptainer is a container platform that doesn't elevate the privileges of a user running the container. This is important as UMIACS runs many multi-tenant hosts (such as Nexus ) and doesn't provide administrative control to users on them. While Docker is popular, the most typical setups require a daemon that has administrative level privileges that makes it not tenable.\n\nApptainer was previously branded as Singularity. You should still be able to run commands on the system with singularity , however you should start migrating to using the apptainer command.\n\nContents\n\n1 Overview\n\n2 Nexus Containers\n\n2.1 Bind Mounts\n\n2.2 Shared Containers\n\n3 Docker Workflow Example\n\nOverview\n\nYou can find out what the current version is that we provide by running the apptainer --version command. If this instead says apptainer: command not found and you are using a UMIACS-supported host, please contact staff and we will ensure that the software is available on the host you are looking for it on.\n\n# apptainer --version\r\napptainer version 1.2.5-1.el8\r\n\nApptainer can run a variety of images including its own format and Docker images . To create images from definition files, you need to have administrative rights. You will need to either use Podman to accomplish this on UMIACS-supported hosts, or alternatively do this on a host that you have full administrative access to (laptop or personal desktop) rather than a UMIACS-supported host.\n\nIf you are going to pull large images, you may run out of space in your home directory. We suggest you run the following commands to setup alternate cache and tmp directories. We are using /scratch0 but you can substitute any large enough local scratch directory, network scratch directory, or project directory you would like.\n\nexport WORKDIR=/scratch0/$USER\r\nexport APPTAINER_CACHEDIR=${WORKDIR}/.cache\r\nexport APPTAINER_TMPDIR=${WORKDIR}/.tmp\r\nmkdir -p $APPTAINER_CACHEDIR\r\nmkdir -p $APPTAINER_TMPDIR\r\n\nWe do suggest you pull images down into an intermediate file ( SIF file) as you then do not have to worry about re-caching the image.\n\n$ apptainer pull cuda12.2.2.sif docker://nvidia/cuda:12.2.2-base-ubi8\r\nINFO: Converting OCI blobs to SIF format\r\nINFO: Starting build...\r\nGetting image source signatures\r\nCopying blob d5d706ce7b29 done\r\nCopying blob b4dc78aeafca done\r\nCopying blob 24a22c1b7260 done\r\nCopying blob 8dea37be3176 done\r\nCopying blob 25fa05cd42bd done\r\nCopying blob a57130ec8de1 done\r\nCopying blob 880a66924cf5 done\r\nCopying config db554d658b done\r\nWriting manifest to image destination\r\nStoring signatures\r\n2022/10/14 10:31:17 info unpack layer: sha256:25fa05cd42bd8fabb25d2a6f3f8c9f7ab34637903d00fd2ed1c1d0fa980427dd\r\n2022/10/14 10:31:19 info unpack layer: sha256:24a22c1b72605a4dbcec13b743ef60a6cbb43185fe46fd8a35941f9af7c11153\r\n2022/10/14 10:31:19 info unpack layer: sha256:8dea37be3176a88fae41c265562d5fb438d9281c356dcb4edeaa51451dbdfdb2\r\n2022/10/14 10:31:20 info unpack layer: sha256:b4dc78aeafca6321025300e9d3050c5ba3fb2ac743ae547c6e1efa3f9284ce0b\r\n2022/10/14 10:31:20 info unpack layer: sha256:a57130ec8de1e44163e965620d5aed2abe6cddf48b48272964bfd8bca101df38\r]",Apptainer can run a variety of images including its own format and Docker images.,"Apptainer can run a variety of images including its own format (Singularity Image Format or .sif files) and Docker images. This means you can use Apptainer to execute both native Apptainer containers as well as those created with the Docker platform, providing flexibility in container management and execution across different formats.",1.0,0.666667
1,What are the guidelines for installing and licensing Adobe Creative Cloud products at UMIACS?,"[Adobe\n\nFrom UMIACS\n\nJump to

In [109]:
import pandas as pd

In [110]:
pd.set_option('display.max_colwidth', None)

In [111]:
resultdf.iloc[6][["question", "ground_truth", "answer"]]

question                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

the automatically generated questions and answers suck (even tho they use chatgpt's good models + ragas even uses 2 models to generate and verify them)
it also took like 150k tokens (???) to generate these 10 questions. over 1 cent per question omg.

I have no idea how useful the metrics are cuz their questions and answers suck, and I didn't try many.